In [3]:
import cv2
import numpy as np
import sqlite3
import bcrypt
import time
from keras_facenet import FaceNet
from mtcnn import MTCNN

# Initialize MTCNN and FaceNet
detection_method = MTCNN()
embed_extract = FaceNet()

# Database connection with exception handling in case the 'database is locked' error apprears
def connect_db_with_exp_handling(retries=5, delay=1):
    for i in range(retries):
        try:
            conn = sqlite3.connect('user_face_data.db')  # Database name
            return conn
        except sqlite3.OperationalError as e:
            if "database is locked" in str(e):
                print(f"Database is locked, retrying... ({i+1}/{retries})")
                time.sleep(delay)
            else:
                raise e
    raise sqlite3.OperationalError("Database is still locked after retries.")

# Clear the existing tables to avoid any conflits,creating new tables if they don't already exists
def initial_db_conn():
    with connect_db_with_exp_handling() as conn:
        c = conn.cursor()
        # Drop the existing tables to clear the database
        c.execute("DROP TABLE IF EXISTS user_info")  
        c.execute("DROP TABLE IF EXISTS face_embeddings")  
        c.execute("DROP TABLE IF EXISTS payment_methods")  
        
        
        # Users table for storing name, email, and hashed password
        c.execute('''CREATE TABLE IF NOT EXISTS user_info 
                     (id INTEGER PRIMARY KEY, name TEXT, email TEXT UNIQUE, password TEXT)''')
        
        # Face embeddings table for storing face, left eye, right eye, and lip embeddings
        c.execute('''CREATE TABLE IF NOT EXISTS face_embeddings 
                     (user_id INTEGER,
                      face_encoding BLOB,
                      left_eye_encoding BLOB,
                      right_eye_encoding BLOB,
                      lip_encoding BLOB,
                      FOREIGN KEY(user_id) REFERENCES user_info(id))''')
        
        # Payment methods table for storing card details
        c.execute('''CREATE TABLE IF NOT EXISTS payment_methods 
                     (id INTEGER PRIMARY KEY, user_id INTEGER, card_number TEXT, bank_name TEXT, 
                      card_type TEXT, expiry_date TEXT, 
                      FOREIGN KEY(user_id) REFERENCES user_info(id))''')
        conn.commit()

# Capture image from webcam
def capture_img():
    video_cap = cv2.VideoCapture(0)
    if not video_cap.isOpened():
        print("Error: Could not open webcam.")
        return None
    ret, frame = video_cap.read()
    video_cap.release()
    if not ret:
        print("Error: Failed to capture image.")
        return None
    return frame

# Detect face along with eyes and lips from the image
def detect_img(img):
    detections = detection_method.detect_faces(img)
    if len(detections) == 0:
        return None, None, None, None, None
    face = detections[0]
    keypoints = face['keypoints']  # Assuming only the first detected face
    left_eye = keypoints['left_eye']
    right_eye = keypoints['right_eye']
    mouth_left = keypoints['mouth_left']
    mouth_right = keypoints['mouth_right']
    return face, left_eye, right_eye, mouth_left, mouth_right

# Extract face region from the image
def ext_face(img, box):
    x, y, w, h = box
    x, y = max(0, x), max(0, y)
    face = img[y:y+h, x:x+w]
    if face.size == 0:
        return None
    face = cv2.resize(face, (160, 160))
    face = face.astype('float32') / 255
    return face

# Extract eye and lip regions from the image
def ext_eye_lip(img, left_eye, right_eye, mouth_left, mouth_right):
    # Define the size for eye and lip regions
    eye_size = 40
    lip_width, lip_height = 60, 40
    
    # Extract left eye region
    left_eye_x, left_eye_y = left_eye
    left_eye_region = img[max(0, left_eye_y - eye_size//2):left_eye_y + eye_size//2,
                         max(0, left_eye_x - eye_size//2):left_eye_x + eye_size//2]
    if left_eye_region.size == 0:
        return None, None, None
    left_eye_region = cv2.resize(left_eye_region, (160, 160))
    left_eye_region = left_eye_region.astype('float32') / 255
    
    # Extract right eye region
    right_eye_x, right_eye_y = right_eye
    right_eye_region = img[max(0, right_eye_y - eye_size//2):right_eye_y + eye_size//2,
                          max(0, right_eye_x - eye_size//2):right_eye_x + eye_size//2]
    if right_eye_region.size == 0:
        return None, None, None
    right_eye_region = cv2.resize(right_eye_region, (160, 160))
    right_eye_region = right_eye_region.astype('float32') / 255
    
    # Extract lip region
    mid_mouth_x = (mouth_left[0] + mouth_right[0]) // 2
    mid_mouth_y = (mouth_left[1] + mouth_right[1]) // 2
    lip_region = img[max(0, mid_mouth_y - lip_height//2):mid_mouth_y + lip_height//2,
                    max(0, mid_mouth_x - lip_width//2):mid_mouth_x + lip_width//2]
    if lip_region.size == 0:
        return None, None, None
    lip_region = cv2.resize(lip_region, (160, 160))
    lip_region = lip_region.astype('float32') / 255
    
    return left_eye_region, right_eye_region, lip_region

# Extract face embedding
def face_embed(face_img):
    if face_img is None:
        return None
    face_list = [face_img]
    embeddings = embed_extract.embeddings(face_list)
    return embeddings[0]

# Extract embeddings for eyes and lips
def eye_lip_embed(left_eye_img, right_eye_img, lip_img):
    if left_eye_img is None or right_eye_img is None or lip_img is None:
        return None, None, None
    left_eye_embed = embed_extract.embeddings([left_eye_img])[0]
    right_eye_embed = embed_extract.embeddings([right_eye_img])[0]
    lip_embed = embed_extract.embeddings([lip_img])[0]
    return left_eye_embed, right_eye_embed, lip_embed

# Register user and store credentials, face, eye, lip embeddings, and payment methods
def register_user(name, email, password, face_embedding, left_eye_embedding, right_eye_embedding, lip_embedding, payment_methods):
    try:
        with connect_db_with_exp_handling() as conn:
            c = conn.cursor()
            hashed_password = bcrypt.hashpw(password.encode('utf-8'), bcrypt.gensalt())  # Hash the password
            # Insert user into user_info table
            c.execute("INSERT INTO user_info (name, email, password) VALUES (?, ?, ?)", 
                      (name, email, hashed_password))
            user_id = c.lastrowid  # Get the inserted user's ID
            
            # Insert embeddings into face_embeddings table
            c.execute('''INSERT INTO face_embeddings 
                         (user_id, face_encoding, left_eye_encoding, right_eye_encoding, lip_encoding) 
                         VALUES (?, ?, ?, ?, ?)''',
                      (user_id, face_embedding.tobytes(), left_eye_embedding.tobytes(),
                       right_eye_embedding.tobytes(), lip_embedding.tobytes()))
            
            # Insert payment methods into payment_methods table
            for payment_method in payment_methods:
                c.execute('''INSERT INTO payment_methods 
                             (user_id, card_number, bank_name, card_type, expiry_date) 
                             VALUES (?, ?, ?, ?, ?)''', 
                          (user_id, payment_method['card_number'], payment_method['bank_name'], 
                           payment_method['card_type'], payment_method['expiry_date']))
            conn.commit()
    except sqlite3.IntegrityError:
        print("Error: The email provided is already registered.")
    except sqlite3.OperationalError as e:
        print(f"Database error: {e}")

# Use 'with' statement for automatic connection management
def load_user_by_email(email):
    with connect_db_with_exp_handling() as conn:
        c = conn.cursor()
        c.execute("SELECT id, name, email, password FROM user_info WHERE email = ?", (email,))
        user = c.fetchone()
    return user

def load_embeddings_by_user_id(user_id):
    with connect_db_with_exp_handling() as conn:
        c = conn.cursor()
        c.execute('''SELECT face_encoding, left_eye_encoding, right_eye_encoding, lip_encoding 
                     FROM face_embeddings WHERE user_id = ?''', (user_id,))
        embeddings = c.fetchone()
    if embeddings:
        face_emb = np.frombuffer(embeddings[0], dtype=np.float32)
        left_eye_emb = np.frombuffer(embeddings[1], dtype=np.float32)
        right_eye_emb = np.frombuffer(embeddings[2], dtype=np.float32)
        lip_emb = np.frombuffer(embeddings[3], dtype=np.float32)
        return face_emb, left_eye_emb, right_eye_emb, lip_emb
    return None, None, None, None

# Load payment methods by user ID
def load_payment_methods_by_user_id(user_id):
    with connect_db_with_exp_handling() as conn:
        c = conn.cursor()
        c.execute('''SELECT id, card_number, bank_name, card_type, expiry_date 
                     FROM payment_methods WHERE user_id = ?''', (user_id,))
        rows = c.fetchall()
    payment_methods = [{'id': row[0], 'card_number': row[1], 'bank_name': row[2], 
                        'card_type': row[3], 'expiry_date': row[4]} for row in rows]
    return payment_methods

# Calculate distance between two embeddings for verification
def calc_dist(embed1, embed2):
    return np.linalg.norm(embed1 - embed2)

# Registration process with face, eye, and lip capture and payment methods input
def register():
    print("Please provide your details to sign up:")
    name = input("Name: ")
    email = input("Email: ")
    password = input("Password: ")

    # Capture face
    print("Now, capture your face:")
    image = capture_img()
    rgb_image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    face, left_eye, right_eye, mouth_left, mouth_right = detect_img(rgb_image)

    if face is None:
        print("No face detected. Please try again.")
        return

    # Extract and embed face
    box = face['box']
    face_image = ext_face(rgb_image, box)
    face_embedding = face_embed(face_image)

    # Extract and embed eyes and lips
    left_eye_img, right_eye_img, lip_img = ext_eye_lip(rgb_image, left_eye, right_eye, mouth_left, mouth_right)

    # Check if any facial feature could not be extracted
    if left_eye_img is None or right_eye_img is None or lip_img is None:
        print("Failed to extract one or more facial features. Please try again.")
        return

    left_eye_embed, right_eye_embed, lip_embed = eye_lip_embed(left_eye_img, right_eye_img, lip_img)

    # Collect payment methods
    payment_methods = []
    while True:
        print("\nAdd a payment method:")
        card_number = input("Enter card number: ")
        bank_name = input("Enter bank name: ")
        card_type = input("Enter card type (credit/debit): ")
        expiry_date = input("Enter expiry date (MM/YY): ")
        payment_methods.append({
            'card_number': card_number,
            'bank_name': bank_name,
            'card_type': card_type,
            'expiry_date': expiry_date
        })
        add_another = input("Do you want to add another payment method? (yes/no): ").lower()
        if add_another != 'yes':
            break

    # Store user credentials, face, iris, and lip embeddings, and payment methods
    register_user(name, email, password, face_embedding, left_eye_embed, right_eye_embed, lip_embed, payment_methods)
    print("Registration successful.")


# Login and verify face, eyes, and lips for transaction
def login():
    print("\n--- User Login ---")
    email = input("Email: ").strip()
    password = input("Password: ").strip()

    user = load_user_by_email(email)
    if user is None:
        print("User not found. Please register.")
        return

    user_id, name, email, stored_password = user

    if bcrypt.checkpw(password.encode('utf-8'), stored_password):
        print(f"\nWelcome {name}!")

        # Ask if the user wants to proceed with a transaction
        transaction = input("Do you want to proceed with a transaction? (yes/no): ").strip().lower()
        if transaction == "yes":
            # Load user's payment methods and select one for the transaction
            payment_methods = load_payment_methods_by_user_id(user_id)
            if len(payment_methods) == 0:
                print("No payment methods found. Please add a payment method.")
                return

            print("\n--- Select a Payment Method ---")
            for idx, method in enumerate(payment_methods):
                print(f"{idx + 1}. {method['card_type'].capitalize()} Card - "
                      f"**** **** **** {method['card_number'][-4:]} "
                      f"(Bank: {method['bank_name']}, Expiry: {method['expiry_date']})")
            
            while True:
                try:
                    selected_method_idx = int(input("Enter the number of the payment method to use: ")) - 1
                    if 0 <= selected_method_idx < len(payment_methods):
                        selected_method = payment_methods[selected_method_idx]
                        break
                    else:
                        print("Invalid selection. Please try again.")
                except ValueError:
                    print("Invalid input. Please enter a number.")

            # Capture and verify face, eyes, and lips for transaction
            if verify_face(user_id):
                print(f"\nPayment with {selected_method['card_type'].capitalize()} Card ending in "
                      f"{selected_method['card_number'][-4:]} successful!")
            else:
                print("\nPayment failed due to unsuccessful verification.")
        else:
            print("Login successful without proceeding to transaction.")
    else:
        print("Incorrect password.")

def verify_face(user_id):
    print("--- Face, Iris, and Lip Verification ---")
    print("Please position your face in front of the webcam for verification...")
    image = capture_img()
    if image is None:
        print("Failed to capture image.")
        return False

    rgb_image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    
    # Detect face and facial keypoints (eyes, mouth)
    face, left_eye, right_eye, mouth_left, mouth_right = detect_img(rgb_image)
    
    if face is None:
        print("No face detected. Please try again.")
        return False
    
    # Extract face region and generate face embedding
    box = face['box']
    face_image = ext_face(rgb_image, box)
    face_embedding = face_embed(face_image)

    # Extract and embed eyes and lips
    left_eye_img, right_eye_img, lip_img = ext_eye_lip(rgb_image, left_eye, right_eye, mouth_left, mouth_right)
    
    # Check if any of the extracted features are None
    if left_eye_img is None or right_eye_img is None or lip_img is None:
        print("Failed to extract one or more facial features.")
        return False

    left_eye_embed, right_eye_embed, lip_embed = eye_lip_embed(left_eye_img, right_eye_img, lip_img)

    # Load stored embeddings for this user
    stored_face_emb, stored_left_eye_emb, stored_right_eye_emb, stored_lip_emb = load_embeddings_by_user_id(user_id)

    if stored_face_emb is None or stored_left_eye_emb is None or stored_right_eye_emb is None or stored_lip_emb is None:
        print("Stored embeddings not found for this user.")
        return False

    # Set a threshold for verification
    threshold = 0.5

    # Compare all embeddings (face, left eye, right eye, lips)
    dist_face = calc_dist(face_embedding, stored_face_emb)
    dist_left_eye = calc_dist(left_eye_embed, stored_left_eye_emb)
    dist_right_eye = calc_dist(right_eye_embed, stored_right_eye_emb)
    dist_lip = calc_dist(lip_embed, stored_lip_emb)

    if (dist_face < threshold and dist_left_eye < threshold and 
        dist_right_eye < threshold and dist_lip < threshold):
        print(f"Face, iris, and lips verified successfully.\n"
              f"Distances: face={dist_face:.4f}, left_eye={dist_left_eye:.4f}, "
              f"right_eye={dist_right_eye:.4f}, lips={dist_lip:.4f}")
        return True

    print("Verification failed.")
    return False


# Main function to run the application
def main():
    initialize_db()  # Initialize the database

    print("\Welcome to the Secure Authentication System")
    while True:
        print("\nPlease choose an option:")
        print("1. Register")
        print("2. Login")
        print("3. Exit")
        choice = input("Enter your choice: ").strip()

        if choice == '1':
            register()
        elif choice == '2':
            login()
        elif choice == '3':
            print("Exiting the application. Goodbye!")
            break
        else:
            print("Invalid choice. Please try again.")

if __name__ == "__main__":
    main()


\Welcome to the Secure Authentication System

Please choose an option:
1. Register
2. Login
3. Exit
Enter your choice: 1
Please provide your details to sign up:
Name: Poulomi
Email: poulomisaha042@gmail.com
Password: Misti@123
Now, capture your face:
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 170ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 169ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 54ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 221ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 5s 5s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 88ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 82ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 86ms/step

Add a payment method:
Enter card number: 5555444422221111
Enter bank name: LLOYDS BANK
Enter card type (credit/debit): debit
Enter expiry d